# Step 2: Model Building & Evaluation
Using the training and test data sets we constructed in the `Code/1_data_ingestion_and_preparation.ipynb` Jupyter notebook, this notebook builds a LSTM network for scenerio described at [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3) to predict failure in aircraft engines. We will store the model for deployment in an Azure web service which we build in the `Code/3_operationalization.ipynb` Jupyter notebook.

In [61]:
# import the libraries
import os

from azureml.core import  (Workspace,Run,VERSION,
                           Experiment,Datastore)
from azureml.core.compute import (AmlCompute, ComputeTarget)
from azureml.exceptions import ComputeTargetException

from azureml.train.dnn import PyTorch
from azureml.train.hyperdrive import *
from azureml.widgets import RunDetails



print('SDK verison', VERSION)

SDK verison 1.0.6


## Azure ML workspace

In [62]:
project_folder = os.getcwd()
exp_name = "deep_pred"

ws = Workspace.from_config()
print('Workspace loaded:', ws.name)

Found the config file in: /home/sasuke/dev/amlsamples/deep_predictive_maintenance/aml_config/config.json
Workspace loaded: vienna


## Load feature data set

We have previously created the labeled data set in the `Code\1_Data Ingestion and Preparation.ipynb` Jupyter notebook and stored it in default data store of the AML workspace.

Here, we call path method that returns an instance to [data reference](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.data_reference.datareference?view=azure-ml-py) which  will be passed to the training script during the run execution.

In [63]:
ds = Datastore.get(ws,'workspaceblobstore')
data_path = "data"
ds_path = ds.path(data_path)
print(ds_path)

$AZUREML_DATAREFERENCE_af29574a359e4b99867c5dcaeecada83


## Compute target

Here, we provision the AML Compute that will be used to execute training script

In [64]:
training_dir = './train'
os.makedirs(training_dir, exist_ok=True)

# choose a name for your cluster
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',
                                                           max_nodes=6)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

Found existing compute target.


## Modelling

The traditional predictive maintenance machine learning models are based on feature engineering, the manual construction of variable using domain expertise and intuition. This usually makes these models hard to reuse as the feature are specific to the problem scenario and the available data may vary between customers. Perhaps the most attractive advantage of deep learning they automatically do feature engineering from the data, eliminating the need for the manual feature engineering step.

When using LSTMs in the time-series domain, one important parameter is the sequence length, the window to examine for failure signal. This may be viewed as picking a `window_size` (i.e. 5 cycles) for calculating the rolling features in the [Predictive Maintenance Template](https://gallery.cortanaintelligence.com/Collection/Predictive-Maintenance-Template-3). The rolling features included rolling mean and rolling standard deviation over the 5 cycles for each of the 21 sensor values. In deep learning, we allow the LSTMs to extract abstract features out of the sequence of sensor values within the window. The expectation is that patterns within these sensor values will be automatically encoded by the LSTM.

Another critical advantage of LSTMs is their ability to remember from long-term sequences (window sizes) which is hard to achieve by traditional feature engineering. Computing rolling averages over a window size of 50 cycles may lead to loss of information due to smoothing over such a long period. LSTMs are able to use larger window sizes and use all the information in the window as input. 

http://colah.github.io/posts/2015-08-Understanding-LSTMs/ contains more information on the details of LSTM networks.

This sample illustrates the LSTM approach to binary classification using a sequence_length of 50 cycles to predict the probability of engine failure within 30 days.

##  Implementation and hyperparameters tuning

Building a Neural Net requires determining the network architecture. In this scenario we will build an LSTM network using Pytorch [estimator](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-train-pytorch).

The hyperparameters tunning of the network is achieved using [Hyperdrive](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-tune-hyperparameters)

In the train directory, the listed below files are used as follow:

 - Utils.py: contains data preparation to read csv files and transform them into lstm ready 3D tensors.
 - network.py: Defines LSTM network in pytorch.
 - train.py: entry script to estimator, contain training script.

In [108]:
%%writefile ./train/network.py

import torch 
import torch.nn as nn

class Network(nn.Module):
    
    def __init__(self,batch_size,input_size, 
                 hidden_size, nb_layers, dropout):
        super(Network, self).__init__()
        
        
        self.hidden_size = hidden_size
        self.nb_layers = nb_layers
        self.dropout = nn.Dropout(dropout)
        self.rnn = nn.LSTM(input_size, hidden_size, 
                           nb_layers, batch_first=True,
                           dropout = dropout)
        self.fc = nn.Linear(hidden_size, 2)
        self.activation = nn.ReLU()
        return (h,c)
    
    def forward(self, sequence):
        
        out,_ = self.rnn(sequence)
        out = self.activation(out)
        out = self.fc(out[:, -1, :])
        return out


Overwriting ./train/network.py


In [109]:
%%writefile ./train/train.py


import torch 
import torch.nn as nn
import torch.utils.data as utils
from network import Network
from sklearn.metrics import precision_score,recall_score,f1_score



def train(X_train,y_train, 
          X_val,y_val, 
          learning_rate,batch_size,
          hidden_size, nb_layers,
          dropout, run):
    
    dataset = utils.TensorDataset(torch.from_numpy(X_train),
                                  torch.from_numpy(y_train)) 
    dataloader = utils.DataLoader(dataset, batch_size = batch_size,
                                  shuffle = True)
    
    val_dataset = utils.TensorDataset(torch.from_numpy(X_val),
                                      torch.from_numpy(y_val))
    val_dataloader = utils.DataLoader(val_dataset)
    
    use_gpu = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    input_size = X_train.shape[2]
    network = Network(batch_size, 
                      input_size,hidden_size,
                      nb_layers,dropout).to(use_gpu)
    
    # Loss and optimizer
    cost_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate)
    
    # Train the model
    for epoch in range(nb_epochs):
        
        
        for i, (X, y) in enumerate(dataloader):
            optimizer.zero_grad()
            
            X = X.to(use_gpu)
            y = y.to(use_gpu)
            y_pred = network(X)
            loss = cost_fn(y_pred, y)

            # Backprop
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                run.log('loss', loss.item())
                
        # end of epoch       
        evaluate(val_dataloader, network, use_gpu)
        network.train()

    return network

def evaluate(dataloader, network, use_gpu, run):
    
    '''
        Evaluate model on validation set
        
        params:
            dataloader: dataloader
            network: model
            use_gpu: device
            run: AML RUN
    '''
    
    
    
    y_pred_lst = []
    y_truth_lst = []
    with torch.no_grad(): 
         
        for i, (X, y) in enumerate(dataloader):
            
                X = X.to(use_gpu)
                output = network(X)
                
                print("SIZE",output.size())
                y_pred = output.max(1, keepdim=True)[1]
                print(y_pred)
                
                y_pred_lst.append(y_pred)
                y_truth_lst.append(y.data.numpy().reshape(-1))
                
                '''y_pred_np = y_pred.to('cpu').data.numpy()
                y_test_np = y.to('cpu').data.numpy()
                y_pred_np = np.argmax(y_pred_np, axis=1)'''
                
        y_pred_np = np.array(y_pred_lst)
        y_truth_np = np.array(y_truth_lst)
        
        precision = precision_score(y_truth_np, y_pred_np)
        recall = recall_score(y_truth_np, y_pred_np)
        f1 = f1_score(y_truth_np, y_pred_np)

        run.log('precision', round(precision,2))
        run.log('recall', round(recall,2))
        run.log('f1', round(f1,2))


Overwriting ./train/train.py


In [110]:
%%writefile ./train/entry.py


import os
import numpy as np
import pandas as pd

from utils import tensorize,to_tensors
from sklearn.model_selection import train_test_split
from azureml.core import Run

if __name__ == '__main__':
    
    print('Pytorch version', torch.__version__)
    
    parser = argparse.ArgumentParser()
    
    parser.add_argument('--epochs', type=int, default=2,
                        help='number of epochs to train')
    parser.add_argument('--learning_rate', type=float,
                        default=1e-3, help='learning rate')
    parser.add_argument('--dropout', type=float,
                        default=.2, help='drop out')
    parser.add_argument('--layers', type=int,
                        default=1, help='number of layers')
    parser.add_argument('--hidden_units', type=int,
                        default=16, help='number of neurons')
    parser.add_argument('--batch_size', type=int,
                        default=16, help='Mini batch size')
    parser.add_argument('--data_path', type=str, 
                        help='path to training-set file')
    parser.add_argument('--output_dir', type=str, 
                        help='output directory')
    
    args = parser.parse_args()
    nb_epochs = args.epochs
    learning_rate = args.learning_rate
    dropout = args.dropout
    data_path = args.data_path
    output_dir = args.output_dir
    nb_layers = args.layers
    batch_size = args.batch_size
    
    hidden_size = args.hidden_units
    batch_size = args.batch_size
    
    SEED = 123
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    
    print("Start training")
    
    print('learning rate', learning_rate)
    print('dropout', dropout)
    print('batch_size', batch_size)
    print('hidden_units', hidden_size)
    
    run = Run.get_context()
    
    os.makedirs(data_path, exist_ok = True)
    training_file = os.path.join(data_path, 'preprocessed_train_file.csv')
    
    X, y = to_tensors(training_file)
    X_train, X_test, y_train, y_test = train_test_split(
                             X, y, test_size=0.15, random_state=SEED)
    

    
    network = train( X_train,y_train, 
                    X_test,y_test, 
                    learning_rate,batch_size,
                    hidden_size,nb_layers,
                    dropout, run)
    
    os.makedirs(output_dir, exist_ok = True)
    model_path = os.path.join(output_dir, 'network.pth')
    
    torch.save(network, model_path)
    run.register_model(model_name = 'network.pth', model_path = model_path)

Overwriting ./train/entry.py


## Estimator

Here, we define the Pytorch estimator.

In [111]:
script_params = {
    '--epochs': 2,
    '--data_path': ds_path,
    '--output_dir': './outputs'
}

estimator = PyTorch(source_directory = training_dir, 
                    conda_packages = ['pandas', 'numpy', 'scikit-learn'],
                    pip_packages = ['torch==1.0.0','torchvision'],
                    script_params=script_params,
                    compute_target=compute_target,
                    entry_script='entry.py',
                    use_gpu=True)

## Hyperparameters tuning using Hyperdrive

Here, we define hyerdrive configuration, as we are interested in true equipement failure, we will configure hyperdrive to maximize precision metric. For completness we will be tracking recall and F1 in the experiment

In [112]:
param_sampling = RandomParameterSampling( {
        'learning_rate': uniform(1e-4, 1e-2),
        'dropout': uniform(.5,.7),
        'layers': choice(1,2),
        'batch_size': choice(16,32,64),
        'hidden_units': choice(4,8,12)
    }
)

termination_policy = BanditPolicy(slack_factor=.1, evaluation_interval=1, delay_evaluation=1)

hd_run_config = HyperDriveRunConfig(estimator=estimator,
                                            hyperparameter_sampling=param_sampling, 
                                            policy=termination_policy,
                                            primary_metric_name='precision',
                                            primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                            max_total_runs=2,
                                            max_concurrent_runs=1)

We submit the exepriment for execution and render the Run execution through the widget

In [113]:
experiment = Experiment(workspace=ws, name=exp_name)
run = experiment.submit(hd_run_config)


In [114]:
RunDetails(run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

# Model registration

Finally, we save the best trained model found by hyperdrive based on the primary metric, we have selected.

In [114]:
best_run = run.get_best_run_by_primary_metric()

model = best_run.register_model(model_name='deep_pdm', model_path='outputs/model')
print(model.name, 'saved')

deep_pdm saved


## Model operationalization


We are now ready to operationalizing the model and deloying the webservice. For testing purposes, we wil use ACI to serve predictions.

For more details on Model deployment workflow in Azure Machine learning service,click [here](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where#deployment-workflow) 

In [86]:
#%%writefile $score/score.py
import os
import torch 
import numpy as np
import pandas as pd
from utils import to_tensors
import torch.utils.data as utils
from azureml.core.model import Model


def init():
    global model
    
    model_path = os.path.join(os.getcwd(),'network.pth')# Model.get_model_path(model_name='deep_pdm')
    print(model_path)
    model = torch.load(model_path, map_location=torch.device('cpu'))
    model.eval()
    print(list(model.modules()))
    

def run(raw_input):
    path = raw_input
    X,_ = to_tensors(path, is_test = True)
    
    output = []
    
    dataset = utils.TensorDataset(torch.from_numpy(X)) 
    dataloader = utils.DataLoader(dataset)
    with torch.no_grad():
        for i, (x,) in enumerate(dataloader):
            score = model(x)
            output.append(score[0,0].item())
        
    print(output)
    

In [87]:
init()

/home/sasuke/dev/amlsamples/deep_predictive_maintenance/network.pth
[Network(
  (dropout): Dropout(p=0.594920723381342)
  (rnn): LSTM(25, 8, num_layers=2, batch_first=True, dropout=0.594920723381342)
  (fc): Linear(in_features=8, out_features=2, bias=True)
  (activation): ReLU()
), Dropout(p=0.594920723381342), LSTM(25, 8, num_layers=2, batch_first=True, dropout=0.594920723381342), Linear(in_features=8, out_features=2, bias=True), ReLU()]


In [88]:
run("./data/preprocessed_test_file.csv")

(93, 50, 25) (93,)
[0.6579362750053406, 0.6827903389930725, 0.6856022477149963, 0.6870995163917542, 0.6892296671867371, 0.6818177700042725, 0.6887280344963074, 0.688127338886261, 0.6774036288261414, 0.6881359219551086, 0.688002347946167, 0.689796507358551, 0.6890905499458313, 0.6787527203559875, 0.6698659062385559, 0.6885688900947571, -1.259244441986084, 0.6891939043998718, 0.689296543598175, -1.2567304372787476, 0.6893431544303894, 0.6885995864868164, 0.6875244975090027, 0.6880383491516113, 0.6770665645599365, -1.2777116298675537, 0.6873858571052551, 0.6873782277107239, -1.2744704484939575, -1.2873883247375488, -1.2401834726333618, 0.6712453961372375, 0.6774820685386658, -1.184152603149414, 0.6149030327796936, -1.202048420906067, 0.6798074245452881, 0.6892711520195007, 0.6633759140968323, 0.22751732170581818, 0.6870855093002319, 0.6890868544578552, -1.2535617351531982, 0.6891904473304749, 0.6876607537269592, 0.6661853194236755, -0.9792339205741882, 0.689935564994812, 0.689170598983764